In [1]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain_community.tools import DuckDuckGoSearchRun, GoogleSearchRun, FileSearchTool


In [12]:
pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
tools = [DuckDuckGoSearchRun()]

In [6]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="Web Search Assistant",
    instructions="You are a marketing assistant that helps people with information about mercedes e-cars.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
    file_ids=None,
)

TypeError: Assistants.create() got an unexpected keyword argument 'file_ids'

## Completion

In [6]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools import DuckDuckGoSearchRun, GoogleSearchRun, FileSearchTool
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain, ConversationChain
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field



### RAG

In [5]:
import sys
sys.path.append('./utils/')
from gpt_tools import *
from sqlite_utils import *

In [6]:
# loader = PyPDFLoader("./mercedes-data.pdf")
# documents  = loader.load()

# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)
# embeddings = OpenAIEmbeddings()
# db = FAISS.from_documents(texts, embeddings)
# retriever = db.as_retriever()

In [7]:
load_profile(USER_ID)

In [8]:
def refresh_user_data(USER_ID):
    
    global TOOLS, persona_rag, similar_history_rag, update_persona
    persona_to_text(USER_ID,'user_persona')
    ## Converting to db
    persona = TextLoader("user_persona.txt")
    persona = persona.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    persona_texts = text_splitter.split_documents(persona)
    persona_db = FAISS.from_documents(persona_texts, embeddings)

    persona_retriever = persona_db.as_retriever()
    persona_rag = create_retriever_tool(
        persona_retriever,
        "user_persona",
        "User persona",
    )


    ## Gathering Neighbors
    similar_account = get_similarity(USER_ID)
    top = sorted(list(similar_account.keys()), key=lambda x: similar_account[x], reverse=True)[:2]
    history_to_text(top,'similar_history')

    similar_history = TextLoader("similar_history.txt")
    similar_history = similar_history.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    similar_history_texts = text_splitter.split_documents(similar_history)
    similar_history_db = FAISS.from_documents(similar_history_texts, embeddings)

    similar_history_retriever = similar_history_db.as_retriever()
    similar_history_rag = create_retriever_tool(
        similar_history_retriever,
        "similar_history",
        "Similar user history",
    )

    TOOLS = [DuckDuckGoSearchRun(), static_rag, persona_rag, update_persona]

class Persona(BaseModel):
    Demographics: str = Field(default=None, description="This encompasses various personal attributes and characteristics that describe an individual or group. It includes information about their gender, age or age group, marital status, and details about their children, such as the number and their ages. It also takes into account their profession, current economic standing, and potential for career growth. The location is also a key factor, covering the type of region they reside in, whether rural or urban, as well as the country they live in. Additionally, it examines their vehicle ownership history, indicating the number and types of vehicles, with a focus on whether they prefer luxury modern executive, compact SUVs or mid-range cars or used cars. It should also include what price of car they prefer. Finally, this description includes the percentage of the overall customer base that this group represents.")
    Customer_Experience: str = Field(default=None, description="This captures the diverse ways customers engage with a brand throughout their journey, highlighting a range of preferences, behaviors, and touchpoints. It assesses whether customers lean toward digital communication or traditional transaction methods. Brand loyalty can play a role in this, leading some customers to request minimal information during their purchases. The preferred mode of communication for service bookings, such as digital channels, phone calls, or direct clicks, is also considered. Additionally, it notes whether customers appreciate pick-up service options, whether are inclined to adopt service contracts when purchasing a car. This approach provides a comprehensive understanding of how customers interact with a brand, ensuring that their unique needs and preferences are met.")
    Psychology: str = Field(default=None, description="This outlines a customer's mindset, focusing on their price sensitivity, passion for cars, and attitude toward spending. It captures whether they follow traditional approaches, are price-sensitive but value quality, and whether they are open to luxury and convenience, showing a willingness to invest in higher-end products. It considers their appetite for financial risk, distinguishing those who prefer reliable, well-known brands from those more adventurous with newer brands. The description also evaluates their buying behavior, noting whether they are impulsive shoppers swayed by societal trends or those who prioritize brand prestige, maintenance, and resale value over simple status symbols. Additionally, it observes whether customers use cars as basic transportation or as a display of status. This information also includes their digital savvy, eco-friendliness, and willingness to pay for sustainable options. It further considers their hobbies and whether they are loyal to established brands or open to exploring newer ones.")
    Marketing_Implications: str = Field(default=None, description="This decribes the positioning strategies of what the customer is attracted to. High-end luxury targets the elite market, while premium with a digital flair appeals to modern, tech-savvy consumers. It might offer a mix of digital and personalised services. The focus on premium emphasizes high quality, and the value strategy attracts budget-conscious customers who want services at a fair price without unnecessary frills.")
    
@tool("update_persona", args_schema=Persona, return_direct=False)
def update_persona(Demographics: str  = None, Customer_Experience: str = None, Psychology: str = None, Marketing_Implications: str  = None) -> None:
    """
    Get information of a user and make a persona by dividing the information into Demographics, Customer Experience, Psychology and Marketing Implications according to the way each of them are described. Used to update the current database and should be called whenever you get info about any parameters. All parameters need not be passed to call this.
    """
    if Demographics is not None:
        append_to_profile_column(USER_ID, 'demographics', Demographics)

    if Customer_Experience is not None:
        append_to_profile_column(USER_ID, 'experience', Customer_Experience)

    if Psychology is not None:
        append_to_profile_column(USER_ID, 'psychology', Psychology)

    if Marketing_Implications is not None:  
        append_to_profile_column(USER_ID, 'marketing', Marketing_Implications)

    ## update the user persona
    refresh_user_data(USER_ID)

    print("Tools updated")


In [9]:
embeddings = OpenAIEmbeddings()
db = FAISS.load_local("mercedes_db/", embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever()
static_rag = create_retriever_tool(
    retriever,
    "mercedes_e_cars_specs",
    "Information about mercedes electric cars and their specifications",
)

In [10]:
persona_rag= None
similar_history_rag = None
TOOLS = [DuckDuckGoSearchRun(), static_rag, persona_rag, similar_history_rag, update_persona]

In [11]:
## Getting the persona
USER_ID = 'vishwa20@outlook.com'
add_user(USER_ID, '123')
refresh_user_data(USER_ID)

User already exists


c:\Users\vishw\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\vishw\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
llm = ChatOpenAI(
    model="gpt-4",
    max_tokens=500,
    max_retries=10,
    )
# Creating the retriever
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, TOOLS, prompt)
agent_executor = AgentExecutor(agent=agent, tools=TOOLS, verbose=True)



In [13]:
MESSAGES = [SystemMessage(content="This will assist customers to buy an electric car that will recommend only Mercedes cars. Your focus is giving concise answers and making a sale. You need to use the file 'user_persona' to identify their needs. These needs need to be updated frequently.\
                          The 'update_persona' function will be called whenever it gets information that needs to be added to any one of the four categories namely, Demographics, Customer Experience, Psychology and Marketing Implications. Anytime a personal information or preference comes, check if it needs to be updated. Then look up the answer in the user_persona document.\
                          The 'summarize_chat' function will be called at the end of the chat to summarize all the information regarding the entire conversation. There will be the suggestions provided by the bot and corresponding to each of them there has to be a reaction recorded as in whether the customer had a positive or negative reaction. Give negative responses to the cars that the customer does not talk about when suggested. Each of the suggestions and the reactions should be in separate lines, preferably in bullet points.\
                          Do not ask many questions. Do not tell the user that the function has been called. Do not mention the source pdf from where it is getting the information.")]

In [15]:
query = "I want an budget friendly car?"
MESSAGES.append(HumanMessage(content=query))
bot_response = agent_executor.invoke({"input": query, "chat_history":MESSAGES})['output']
MESSAGES.append(AIMessage(content=bot_response))



> Entering new AgentExecutor chain...

Invoking: `user_persona` with `{'query': 'budget friendly'}`


Email: vishwa20@outlook.com
Demographics: None

Psychology: None

Experience: None

Marketing: None

History: None
Invoking: `mercedes_e_cars_specs` with `{'query': 'budget friendly'}`


- Weight Unladen: 2630 kg  
  - Seats: 8 people  
- **Miscellaneous:**  
  - Cargo Volume: 990 L (Max 4630 L)  
  - Towing: Not available  
  - Heat pump: Not available  
  - Roof Rails: Yes  
  
**Pricing:**  
- **Price:**  
  - United Kingdom: Not Available  
  - The Netherlands: €84,773  
  - Germany: €72,043  
- **Available to Order:**  
  - United Kingdom: Not Available  
  - The Netherlands: Since February 2024  
  - Germany: Since February 2024  
  
**Energy & Fuel Efficiency:**  
- **Energy Consumption:**  
  - EVDB Real Range: 320 km  
  - Vehicle Consumption: 281 Wh/km  
- **WLTP Ratings:**  
  - Range: 381 km  
  - Rated Consumption: 259 Wh/km  
  
**Charging Information:**  
- **Home and 

### SQLite Adding Information

In [178]:
MESSAGES

[SystemMessage(content="This will assist customers to buy an electric car that will recommend only Mercedes cars. Your focus is giving concise answers and making a sale. You need to use the file 'user_persona' to identify their needs.                          The 'update_persona' function will be called whenever it gets information that needs to be added to any one of the four categories namely, Demographics, Customer Experience, Psychology and Marketing Implications.                           The 'summarize_chat' function will be called at the end of the chat to summarize all the information regarding the entire conversation. There will be the suggestions provided by the bot and corresponding to each of them there has to be a reaction recorded as in whether the customer had a positive or negative reaction. Give negative responses to the cars that the customer does not talk about when suggested. Each of the suggestions and the reactions should be in separate lines, preferably in bullet

In [10]:
import sqlite3


def init_db():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS profiles (
            email_id TEXT PRIMARY KEY,
            password TEXT NOT NULL,
            demographics TEXT,
            psychology TEXT,
            experience TEXT,
            marketing TEXT,
            history TEXT
        );
    ''')
    conn.commit()
    conn.close()

def add_user(email_id, password):
    try:
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute('''
            INSERT INTO profiles (email_id, password) VALUES (?, ?)
        ''', (email_id, password))
        conn.commit()
        conn.close()
    except:
        print("User already exists")
        conn.commit()
        conn.close()

def load_profile(email_id):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''
        SELECT demographics, psychology, experience, marketing, history FROM profiles WHERE email_id = ?
    ''', (email_id,))
    profile = c.fetchone()
    conn.close()
    return profile

def append_to_profile_column(email_id, column_name, data):
    '''
    Appends new information to one of emographics, psychology, experience, marketing columns
    '''
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    ## Get the current value in the column
    c.execute('''
        SELECT {} FROM profiles WHERE email_id = ?
    '''.format(column_name), (email_id,))

    current_value = c.fetchone()[0]
    if current_value is None:
        new_value = data
    else:
        new_value = current_value + "/n" + data
    c.execute('''
        UPDATE profiles SET {} = ? WHERE email_id = ?
    '''.format(column_name), (new_value, email_id))
    conn.commit()
    conn.close()

def display_profile_table():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''
        SELECT * FROM profiles
    ''')
    rows = c.fetchall()
    conn.close()
    return rows

def delete_user(email_id):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''
        DELETE FROM profiles WHERE email_id = ?
    ''', (email_id,))
    conn.commit()
    conn.close()

def get_all_emails(exclude):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''
        SELECT email_id FROM profiles WHERE email_id != ?
    ''', (exclude,))
    emails = c.fetchall()
    conn.close()
    return emails

def does_email_exist(email):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''
        SELECT email_id FROM profiles WHERE email_id = ?
    ''', (email,))
    rows = c.fetchall()
    conn.close()
    return len(rows) > 0

def persona_to_text(email, file_name):
    '''
    Save the profile to a well structured text file .txt
    '''
    profile = load_profile(email)
    text = f"Email: {email}\n"
    text += f"Demographics: {profile[0]}\n"
    text += f"Psychology: {profile[1]}\n"
    text += f"Experience: {profile[2]}\n"
    text += f"Marketing: {profile[3]}\n"
    text += f"History: {profile[4]}\n"

    ## Write to a text file
    with open(f"{file_name}.txt", "w") as f:
        f.write(text)

def history_to_text(emails, file_name):
    '''
    Save the history of all emails to a text file
    '''
    text = ""
    for email in emails:
        profile = load_profile(email[0])
        text += f"Email: {email[0]}\n"
        text += f"History: {profile[4]}\n"
        text += "\n"

    ## Write to a text file
    with open(f"{file_name}.txt", "w") as f:
        f.write(text)

In [2]:
add_user('Viola@temp.com', '123')

User already exists


In [3]:
## Create a new user with email_id Franz@temp.com and password 123

# init_db()
# add_user('Sally@temp.com', '123')

## Add this dictionary to Franz
customer_info = {
    "demographics": "Constitutes 33% of the customer base, with strong numbers in Sweden, Switzerland, and Germany.\nLargest and oldest segment, averaging 47 years.\nPredominantly male.\nAmong the lower income brackets.\nFewer children in households, possibly due to grown children.\nResides mainly in rural areas and smaller towns.\nHighest proportion of retirees.\nPrefers used cars, particularly from the mid-range segment, and keeps them the longest.",
                     
    "psychology": "Lowest brand loyalty, often opts for local independent service.\nValues personal relationships, including with his service provider.\nHighly price-sensitive but will pay for quality and service.\nSeeks specialized service staff and personal connections.\nInterested in car maintenance and resale value, but not as a status symbol.\nSelectively digitally savvy.\nCar is seen primarily as a means of transportation, not a status symbol.",
                  
    "experience": "Digitally engaged, especially in price comparison.\nAmong the highest for not seeking information at all.\nPrefers telephone communication for service bookings.",
                           
    "marketing": "Positioning Strategy: Value.\nCan be a loyal customer if engaged, but challenging to attract due to long-standing personal relationships.\nFocus on delivering quality and service at a fair price, without unnecessary frills."
}

for key, value in customer_info.items():
    append_to_profile_column('Viola@temp.com',key,value)
                             

### Embedding Models

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain.evaluation import load_evaluator
import numpy as np
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

hf_evaluator = load_evaluator("embedding_distance", embeddings=embeddings_model)


In [7]:
def get_similarity(email):
    target = load_profile(email)[:-1]
    email_lists = [i[0] for i in get_all_emails(email)]
    lookups =  {i:load_profile(i)[:-1] for i in email_lists}
    similarities = {}
    for email,lookup in lookups.items():
        similarity = []
        for t, l in zip(target, lookup):
            if t is not None and l is not None:
                similarity.append(hf_evaluator.evaluate_strings(prediction=t, reference=l)['score'])
        similarities[email] = np.mean(similarity)

    return similarities

In [8]:
get_similarity('Peter@temp.com')

{'Franz@temp.com': 0.11153727355562928,
 'Sally@temp.com': 0.11670364148574888,
 'Viola@temp.com': 0.12527448644034478}

In [9]:
display_profile_table()

[('Sally@temp.com',
  '123',
  'Represents 14% of the global customer base, with a strong presence in China, France, and Switzerland.\nA younger demographic averaging 32 years old.\nSlightly more males but with a significant female representation.\nNotably, many are childless couples with high disposable income.\nHigh earners, ranking third among all personas.\nA significant urban presence, especially in megacities.\nMajority are in management roles.\nPrefers new entry-level luxury vehicles, with a typical choice being a modern compact executive car.',
  'No extreme psychographic traits but shows a lean towards luxury and convenience.\nImpulsive buyer with a low inclination to save.\nConforms to societal expectations and trends.\nWilling to take financial risks.\nSeeks social validation for purchases, less driven by intrinsic sustainability values but will pay for sustainable products.\nHighly loyal and trusting towards the brand.',
  'Influenced by communication and social circles to 

### Import 

In [1]:
import sys
sys.path.append('./utils/')
from gpt_tools import *

In [2]:
get_similarity('Peter@temp.com')

{'Franz@temp.com': 0.11149944428203365,
 'Sally@temp.com': 0.11670364148574888,
 'Viola@temp.com': 0.125276939241023}

## Import Try

In [1]:
import sys
sys.path.append('./utils/')
from gpt_tools import *

In [2]:
USER_ID = 'vishwa20@outlook.com'
update_user_id(USER_ID)
add_user(USER_ID, '123')
refresh_user_data(USER_ID)

c:\Users\vishw\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\vishw\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [3]:
chat("Which are some other alternatives?")

[DuckDuckGoSearchRun(), Tool(name='mercedes_e_cars_specs', description='Information about mercedes electric cars and their specifications', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DF325C2840>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DF33186850>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001DF325C3740>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DF33186850>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')), Tool(name='user_persona', description='User persona', args_schema=<clas

'Could you please specify the context? Are you inquiring about alternative electric cars to a particular Mercedes model or just in general?'

In [6]:
chat("Thanks, these seem great. I will consider them.")

[DuckDuckGoSearchRun(), Tool(name='mercedes_e_cars_specs', description='Information about mercedes electric cars and their specifications', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DF325C2840>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DF33186850>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001DF325C3740>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DF33186850>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')), Tool(name='user_persona', description='User persona', args_schema=<clas

"I'm glad you found the suggestions helpful! If you have any more questions or need further assistance, feel free to ask. Happy car shopping!"

In [7]:
delete_user(USER_ID)